In [9]:
# Relevant module imports and installs
import pandas as pd
!pip install pulp
import pulp as plp
import sys 
import os
from collections import defaultdict


[notice] A new release of pip is available: 23.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
# Get the absolute path to the directory containing the Python file
module_path = os.path.abspath(os.path.join('..', '..'))

# Add the directory to sys.path
if module_path not in sys.path:
    sys.path.append(module_path)

# Now you can import the module
from projections import generate_projections, get_gameweek_projections

projections_data = get_gameweek_projections(generate_projections(), 1)

display(projections_data)

,ID,Name,Team,Position,Cost,Predicted_Points,Opponent
0,1,Fábio Vieira,Arsenal,Midfielder,5.5,0.02,Wolves (H)
1,2,G.Jesus,Arsenal,Forward,7.0,0.66,Wolves (H)
2,3,Gabriel,Arsenal,Defender,6.0,5.49,Wolves (H)
3,4,Havertz,Arsenal,Forward,8.0,5.84,Wolves (H)
4,6,J.Timber,Arsenal,Defender,5.5,0.00,Wolves (H)
...,...,...,...,...,...,...,...
558,590,Osula,Newcastle,Forward,5.0,0.09,Southampton (H)
559,591,Todibo,West Ham,Defender,4.5,0.00,Aston Villa (H)
560,592,J.Araujo,Bournemouth,Defender,4.5,0.00,Nott'm Forest (A)
561,593,De Ligt,Man Utd,Defender,5.0,0.00,Fulham (H)


# 2024/25 GW1 Challenge: Unlimited Budget, 5-a-side

In [12]:
# Get the number of players and their list of ids
player_ids = projections_data['ID'].tolist()
player_count = len(player_ids)

# Set up the problem
model = plp.LpProblem("fpl-gw1-challenge", plp.LpMaximize)

# Define the decision variables
lineup = [
    plp.LpVariable(f"lineup_{i}", lowBound=0, upBound=1, cat="Integer")
    for i in player_ids
]

# Define captain variables
captain = [
    plp.LpVariable(f"captain_{i}", lowBound=0, upBound=1, cat="Integer")
    for i in player_ids
]

# Set the objective function (the number of points scored by the team, with captain's points doubled)
model += plp.lpSum([lineup[i] * projections_data.loc[i, 'Predicted_Points'] for i in range(player_count)]) + \
         plp.lpSum([captain[i] * projections_data.loc[i, 'Predicted_Points'] for i in range(player_count)])

# Constraints

# Total number of players = 5
model += plp.lpSum(lineup) == 5

# Exactly one captain
model += plp.lpSum(captain) == 1

# Captain must be in the lineup
for i in range(player_count):
    model += captain[i] <= lineup[i]

# At least 1 Goalkeeper
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Goalkeeper']) >= 1

# At least 1-2 Defenders
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Defender']) >= 1
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Defender']) <= 2

# 1-2 Midfielders
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Midfielder']) >= 1
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Midfielder']) <= 2

# 0-2 Forwards
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Forward']) <= 2

# Solve the problem
model.solve()

# Function to print players by position
def print_players_by_position(players_dict):
    total_points = 0
    for position in ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']:
        if position in players_dict:
            
            print(f"\n{position}:")
            for player in players_dict[position]:
                captain_str = " (C)" if player['Captain'] else ""
                points = player['Predicted_Points'] * (2 if player['Captain'] else 1)
                print(f"  {player['Name']}{captain_str} - {player['Team']} - Predicted Points: {points}")
                total_points += points
    print(f"\nTotal Predicted Points: {total_points}")

# Print the results
print("Status:", plp.LpStatus[model.status])

print("\nOptimal Lineup:")
selected_players = defaultdict(list)
for i in range(player_count):
    if lineup[i].value() == 1:
        player = projections_data.loc[i]
        selected_players[player['Position']].append({
            'Name': player['Name'],
            'Team': player['Team'],
            'Predicted_Points': player['Predicted_Points'],
            'Captain': captain[i].value() == 1
        })

print_players_by_position(selected_players)

Status: Optimal

Optimal Lineup:

Goalkeeper:
  Raya - Arsenal - Predicted Points: 4.59

Defender:
  Gabriel - Arsenal - Predicted Points: 5.49

Midfielder:
  M.Salah (C) - Liverpool - Predicted Points: 14.94

Forward:
  Haaland - Man City - Predicted Points: 7.2
  Isak - Newcastle - Predicted Points: 7.19

Total Predicted Points: 39.41
